In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
from collections import defaultdict
from collections import Counter
import matplotlib.pyplot as plt
import pickle


import psutil
psutil.cpu_count()
p = psutil.Process()
p.cpu_affinity([0,1,2,3])

os.chdir(r'H:\all\RL_Shrinkage_2024')
from helpers import helper_functions as hf
from ONE_YR.NonLinear_Shrinkage import regression_evaluation_funcs as re_hf
from helpers import eval_function_new
from helpers import eval_funcs


past_ret_mat contains the permnos of the weights for each rebalancing date. This is needed for the Turnover, as stocks get in/out of the portfolio between rebalancings.

In [50]:
rets_permnos_path = fr"H:\all\RL_Shrinkage_2024\ONE_YR\preprocessing\rets_permnos_1Y"
with open(rf"{rets_permnos_path}/returns_full_1Y_p{PF_SIZE}.pickle", 'rb') as f:
    rets_full = pd.read_pickle(f)
with open(rf"{rets_permnos_path}/permnos_1Y_p{PF_SIZE}.pickle", 'rb') as f:
    permnos = pd.read_pickle(f)


reb_date_1 = permnos.index[0]
add_idx = np.where(rets_full.index == reb_date_1)[0][0]
assert add_idx == 259, "ADD IDX IS NOT 259, CHECK DATES"
for idx in range(5313, 10353, 21):
    shrk_res = []
    date = permnos.index[idx]
    past_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + 259 - 252: idx + 259, :]
    past_ret_mat = past_ret_mat.fillna(0)
    past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
    fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + 259: idx + 259 + 21, :]

    past_ret_mat.columns
    break

In [23]:
def calculate_turnover_metrics(weights_with_idx, val_indices):
    running_sum_turnover = 0
    running_sum_gross_leverage = np.sum(np.abs(weights_with_idx.iloc[: -1].values))  # last rebdate, since its excluded from the for loop
    running_sum_prop_leverage = np.sum(weights_with_idx.iloc[:, -1].values < 0) # last rebdate, since its excluded from the for loop
    for i in range(len(val_indices) - 1):
        df_tmp = pd.concat([weights_with_idx.iloc[:, i], weights_with_idx.iloc[:, i+1]], axis=1).fillna(0)
        df_tmp.columns = ['col1', 'col2']
        running_sum_turnover += np.sum(np.abs((df_tmp['col1'] - df_tmp['col2']).values))
        running_sum_gross_leverage += np.sum(np.abs(weights_with_idx.iloc[:, i].values))
        running_sum_prop_leverage += np.sum(weights_with_idx.iloc[:, i].values < 0)

    running_sum_turnover = np.round(running_sum_turnover / (len(val_indices) - 1), 6)
    running_sum_gross_leverage = np.round(running_sum_gross_leverage / len(val_indices), 6)
    running_sum_prop_leverage = np.round(running_sum_prop_leverage / ( len(val_indices) * weights_with_idx.iloc[:, -1].shape[0]), 6)
    
    return (running_sum_turnover, running_sum_gross_leverage, running_sum_prop_leverage)

In [28]:
def helper_pf_metrics(weights_with_idx, val_indices):
    running_sum_turnover = 0
    running_sum_gross_leverage = np.sum(np.abs(weights_with_idx[-1].values))
    running_sum_prop_leverage = np.sum(weights_with_idx[-1].values < 0)
    for i in range(len(val_indices) - 1):
        df_tmp = pd.concat([weights_with_idx[i], weights_with_idx[i+1]], axis=1).fillna(0)
        df_tmp.columns = ['col1', 'col2']
        running_sum_turnover += np.sum(np.abs((df_tmp['col1'] - df_tmp['col2']).values))
        running_sum_gross_leverage += np.sum(np.abs(weights_with_idx[i].values))
        running_sum_prop_leverage += np.sum(weights_with_idx[i].values < 0)
    running_sum_turnover = np.round(running_sum_turnover / (len(val_indices) - 1), 6)
    running_sum_gross_leverage = np.round(running_sum_gross_leverage / len(val_indices), 6)
    running_sum_prop_leverage = np.round(running_sum_prop_leverage / ( len(val_indices) * weights_with_idx[-1].shape[0]), 6)
    return (running_sum_turnover, running_sum_gross_leverage, running_sum_prop_leverage)


def grid_eval_fixed_shrkges_pf_metrics(fut_ret_mats, past_ret_mats, val_indices, weights):
    TO = f"TO & "
    GL = f"GL & "
    PL = f"PL & "
    for cur_shrkg in weights.shape[0]:
        weights_matrix = []
        for i in range(len(val_indices)):
            shrk, sample, target = BASE_ESTIMATOR(past_ret_mats[val_indices[i]])
            covmat = cur_shrkg * target + (1-cur_shrkg) * sample
            weights = hf.calc_global_min_variance_pf(covmat)
            weights_matrix.append(pd.DataFrame(weights, index=fut_ret_mats[val_indices[i]].columns.tolist()))
        res = helper_pf_metrics(weights_matrix, val_indices)
        TO += f"{res[0]} & "
        GL += f"{res[1]} & "
        PL += f"{res[2]} & "
    return (TO, GL, PL)

## Get Results for Linear Models
(Linear and RL-L)

In [124]:
linear_res_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results"
linear_model_results = {}
l_rl_model_results = {}

for PF_SIZE in [30, 50, 100, 225, 500]:
    c1p_base_weights = pd.read_csv(linear_res_path + f"\p{PF_SIZE}\c1p_p{PF_SIZE}_weights.csv", index_col=0)
    l_rl_weights = pd.read_csv(linear_res_path + f"\p{PF_SIZE}\oos_linear_model_weights_p{PF_SIZE}.csv", index_col=0)

    rets_permnos_path = fr"H:\all\RL_Shrinkage_2024\ONE_YR\preprocessing\rets_permnos_1Y"
    with open(rf"{rets_permnos_path}/returns_full_1Y_p{PF_SIZE}.pickle", 'rb') as f:
        rets_full = pd.read_pickle(f)
    with open(rf"{rets_permnos_path}/permnos_1Y_p{PF_SIZE}.pickle", 'rb') as f:
        permnos = pd.read_pickle(f)


    # in the above weights dataframes the rebalancing dates go along the columns
    reb_date_1 = permnos.index[0]
    add_idx = np.where(rets_full.index == reb_date_1)[0][0]
    assert add_idx == 259, "ADD IDX IS NOT 259, CHECK DATES"
    C1P = {}
    L_RL = {}
    for idx1, idx2 in enumerate(range(5040, 10353, 21)):
        shrk_res = []
        date = permnos.index[idx2]
        past_ret_mat = rets_full[permnos.iloc[idx2]].iloc[idx2 + 259 - 252: idx2 + 259, :]
        past_ret_mat = past_ret_mat.fillna(0)
        past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
        #fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + 259: idx + 259 + 21, :]
        
        tmp_c1p = {}
        tmp_l_rl = {}
        for permnos_idx in range(PF_SIZE):
            tmp_c1p[past_ret_mat.columns[permnos_idx]] = c1p_base_weights.iloc[permnos_idx, idx1]
            tmp_l_rl[past_ret_mat.columns[permnos_idx]] = l_rl_weights.iloc[permnos_idx, idx1]
        C1P[date] = tmp_c1p
        L_RL[date] = tmp_l_rl
    
    C1P = pd.DataFrame(C1P)
    L_RL = pd.DataFrame(L_RL)

    C1P = C1P.fillna(0)
    weights_list = [C1P.iloc[:, i] for i in range(C1P.shape[1])]
    TO, _, _ = helper_pf_metrics(weights_list, list(range(len(weights_list))))
    weights_list = [c1p_base_weights.iloc[:, i] for i in range(c1p_base_weights.shape[1])]
    _, GL, PL = helper_pf_metrics(weights_list, list(range(len(weights_list))))
    linear_model_results[PF_SIZE] = [TO, GL, PL]

    L_RL = L_RL.fillna(0)
    weights_list = [L_RL.iloc[:, i] for i in range(L_RL.shape[1])]
    TO, _, _ = helper_pf_metrics(weights_list, list(range(len(weights_list))))
    weights_list = [l_rl_weights.iloc[:, i] for i in range(l_rl_weights.shape[1])]
    _, GL, PL = helper_pf_metrics(weights_list, list(range(len(weights_list))))
    l_rl_model_results[PF_SIZE] = [TO, GL, PL]

Linear Model Results

In [125]:
pd.DataFrame(linear_model_results, index=["TO","GL","PL"]).T.round(2)

,TO,GL,PL
30,0.61,2.05,0.35
50,0.94,2.61,0.38
100,1.60,3.62,0.43
225,3.03,5.59,0.45
500,3.10,5.67,0.44


L-RL Results

In [126]:
pd.DataFrame(l_rl_model_results, index=["TO","GL","PL"]).T.round(2)

,TO,GL,PL
30,0.09,3.82,0.41
50,0.16,3.86,0.41
100,0.29,3.62,0.41
225,0.54,3.25,0.39
500,1.07,2.96,0.38


### RL-NL Models

In [137]:
nl_res_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\NonLinear_Shrinkage\results"
qis_base_results = {}
qis_rl = {}

for PF_SIZE in [30, 50, 100, 225, 500]:
    qis_base_weights = pd.read_csv(nl_res_path + f"\QIS_base_weights_p{PF_SIZE}.csv", index_col=0)
    qis_rl_weights = pd.read_csv(nl_res_path + f"\model_NL_weights_p{PF_SIZE}.csv", index_col=0)

    rets_permnos_path = fr"H:\all\RL_Shrinkage_2024\ONE_YR\preprocessing\rets_permnos_1Y"
    with open(rf"{rets_permnos_path}/returns_full_1Y_p{PF_SIZE}.pickle", 'rb') as f:
        rets_full = pd.read_pickle(f)
    with open(rf"{rets_permnos_path}/permnos_1Y_p{PF_SIZE}.pickle", 'rb') as f:
        permnos = pd.read_pickle(f)


    # in the above weights dataframes the rebalancing dates go along the columns
    reb_date_1 = permnos.index[0]
    add_idx = np.where(rets_full.index == reb_date_1)[0][0]
    assert add_idx == 259, "ADD IDX IS NOT 259, CHECK DATES"
    QIS_BASE = {}
    QIS_RL = {}
    for idx1, idx2 in enumerate(range(5040, 10353, 21)):
        shrk_res = []
        date = permnos.index[idx2]
        past_ret_mat = rets_full[permnos.iloc[idx2]].iloc[idx2 + 259 - 252: idx2 + 259, :]
        past_ret_mat = past_ret_mat.fillna(0)
        past_ret_mat = past_ret_mat.sub(past_ret_mat.mean())
        #fut_ret_mat = rets_full[permnos.iloc[idx]].iloc[idx + 259: idx + 259 + 21, :]
        
        tmp_qis_base = {}
        tmp_qis_rl = {}
        for permnos_idx in range(PF_SIZE):
            tmp_qis_base[past_ret_mat.columns[permnos_idx]] = qis_base_weights.iloc[permnos_idx, idx1]
            tmp_qis_rl[past_ret_mat.columns[permnos_idx]] = qis_rl_weights.iloc[permnos_idx, idx1]
        QIS_BASE[date] = tmp_qis_base
        QIS_RL[date] = tmp_qis_rl
    
    QIS_BASE = pd.DataFrame(QIS_BASE)
    QIS_RL = pd.DataFrame(QIS_RL)

    QIS_BASE = QIS_BASE.fillna(0)
    weights_list = [QIS_BASE.iloc[:, i] for i in range(QIS_BASE.shape[1])]
    TO, _, _ = helper_pf_metrics(weights_list, list(range(len(weights_list))))
    weights_list = [qis_base_weights.iloc[:, i] for i in range(qis_base_weights.shape[1])]
    _, GL, PL = helper_pf_metrics(weights_list, list(range(len(weights_list))))
    qis_base_results[PF_SIZE] = [TO, GL, PL]

    QIS_RL = QIS_RL.fillna(0)
    weights_list = [QIS_RL.iloc[:, i] for i in range(QIS_RL.shape[1])]
    TO, _, _ = helper_pf_metrics(weights_list, list(range(len(weights_list))))
    weights_list = [qis_rl_weights.iloc[:, i] for i in range(qis_rl_weights.shape[1])]
    _, GL, PL = helper_pf_metrics(weights_list, list(range(len(weights_list))))
    qis_rl[PF_SIZE] = [TO, GL, PL]

In [141]:
pd.DataFrame(linear_model_results, index=["TO","GL","PL"]).T.round(2)

,TO,GL,PL
30,0.61,2.05,0.35
50,0.94,2.61,0.38
100,1.60,3.62,0.43
225,3.03,5.59,0.45
500,3.10,5.67,0.44


In [140]:
pd.DataFrame(l_rl_model_results, index=["TO","GL","PL"]).T.round(2)

,TO,GL,PL
30,0.09,3.82,0.41
50,0.16,3.86,0.41
100,0.29,3.62,0.41
225,0.54,3.25,0.39
500,1.07,2.96,0.38


qis base

In [143]:
pd.DataFrame(qis_base_results, index=["TO","GL","PL"]).T.round(2)

,TO,GL,PL
30,0.80,2.20,0.33
50,0.86,2.36,0.35
100,1.02,2.60,0.37
225,1.90,4.02,0.42
500,3.55,4.99,0.43


qis nl rl

In [142]:
pd.DataFrame(qis_rl, index=["TO","GL","PL"]).T.round(2)

,TO,GL,PL
30,0.80,2.15,0.33
50,0.86,2.30,0.34
100,1.11,2.67,0.38
225,1.79,3.97,0.42
500,3.59,5.09,0.43
